In [2]:
#TSception testing
from torcheeg.models import EEGNet
import pytorch_lightning as pl
from pytorch.data_setup.DataModule import DataModule
from pathlib import Path
from pytorch.models.TSception import TSception

#DataModule:
data_dir = Path("C:/Users/s_gue/Desktop/master_project/sven-thesis/data/preprocessed/wet")
#data_dir = Path("C:/Users/s_gue/Desktop/master_project/sven-thesis/data/preprocessed/wet/sub-P001_ses-S002_task-Default_run-001_eeg")
dm = DataModule(data_dir, hold_out = True, batch_size = 32, num_workers = 0, seed = 42)

model = TSception(
    num_T = 9, 
    num_S = 6, 
    hidden = 16,
    num_classes = 20, 
    input_size = [8, 500], 
    sampling_rate = 250, 
    dropout_rate = 0.3,
    lr = 5e-4
    )

#Trainer:
# trainer = pl.Trainer(max_epochs=200, accelerator="gpu")
# trainer.fit(model, dm)

In [4]:
#EEGNet testing
from torcheeg.models import EEGNet
import pytorch_lightning as pl
from pytorch.data_setup.DataModule import DataModule
from pathlib import Path
from pytorch.models.EEGNet import EEGNetv4

#DataModule:
data_dir = Path("C:/Users/s_gue/Desktop/master_project/sven-thesis/data/preprocessed/wet")
#data_dir = Path("C:/Users/s_gue/Desktop/master_project/sven-thesis/data/preprocessed/wet/sub-P001_ses-S002_task-Default_run-001_eeg")
dm = DataModule(data_dir, hold_out = True, batch_size = 32, num_workers = 0, seed = 42)

model = EEGNetv4(      
            in_chans=8,          #Number of electrodes
            n_classes=20,        
            drop_prob=0.25,       
            kernel_length=128,   #Filter size of block 1
            input_window_samples=500,
            final_conv_length="auto",
            pool_mode="mean",
            F1=8,                #Filter number of block 1
            F2=16,               #Filter number of block 2
            D=2,                 #The depth multiplier (number of spatial filters)     
            third_kernel_size=(8, 4),
            momentum = 0.0,
            lr = 1e-3,
            weight_decay = 0.001
            )      

#Trainer:
trainer = pl.Trainer(max_epochs=200, accelerator="gpu")
trainer.fit(model, dm)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3070 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                 | Type                 | Params
---------------------------------------------------------------
0  | loss                 | NLLLoss              | 0     
1  | acc                  | MulticlassAccuracy   | 0     
2  | ensuredims           | Ensure4d             | 0     
3  | dimshuffle           | Expression           | 0     
4  | conv_temporal        | Conv2d               | 1.0 K 
5  | bnorm_tempora

Epoch 114:   1%|          | 1/105 [00:00<00:02, 35.76it/s, v_num=20, val_loss=4.320, val_acc=0.0548, train_loss=1.520, train_acc=0.512]  

In [ ]:
#Debug wandb sweep (Normal debugger not working with wandb)
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
import wandb
import yaml
from pytorch.data_setup.DataModule import DataModule
from pytorch.models.EEGNet import EEGNetv4

def read_config(config_path: str):
    with open(config_path) as file:
        config = yaml.load(file, Loader=yaml.FullLoader)
    return config

read_config(config_path = "../pytorch/configs/config_EEGNet.yaml")

def main(config = None):
    # Initialize a new wandb run
    wandb.init(config=config)
    # Initialize data module
    dm = DataModule(**wandb.config["datamodule"])
    
    # Initialize model
    model = EEGNetv4(**wandb.config["model"]) 
    print(model)
    # Create a ModelCheckpoint callback
    checkpoint_callback = ModelCheckpoint(
        monitor="val_acc",
        mode="max",
        filename="best-model-{epoch:02d}-{val_acc:.2f}",
        save_top_k=1,
        save_weights_only=True,
        verbose=True,
    )
    lr_monitor = LearningRateMonitor(logging_interval="epoch")

    # Initialize trainer
    trainer = pl.Trainer(
        max_epochs = wandb.config.trainer["max_epochs"],
        logger = pl.loggers.WandbLogger(save_dir="C:/Users/s_gue/Desktop/master_project/sven-thesis/results/wandb_logs/"),
        callbacks = [checkpoint_callback, lr_monitor],
        default_root_dir="C:/Users/s_gue/Desktop/master_project/sven-thesis/results/checkpoints", 
        #pl.callbacks.EarlyStopping(monitor="val_acc")
    )

    # Train model 
    #model.stepsize = np.around(train_set.__len__()*0.8/config["batch_size"]) 
    trainer.fit(model = model, datamodule = dm)

    # Test model
    #trainer.test(datamodule = dm)
    #print("AFTER_TEST")

    wandb.run.finish()

# Initialize new sweep 
sweep_config = read_config(config_path = "../pytorch/configs/config_EEGNet.yaml")
sweep_id = wandb.sweep(sweep_config, project=sweep_config["name"])

# Run sweep
wandb.agent(sweep_id, function=main)